Machine Learning 

1.We will now try to predict per capita crime rate in the Boston data set.

In [71]:
!pip install ISLP

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [72]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from itertools import combinations
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_openml

(a)	Try out some of the regression methods explored in this chapter, such as best subset selection, the lasso, ridge regression, and PCR. Present and discuss results for the approaches that you consider. 

In [73]:

boston = fetch_openml(name="boston", version=1, as_frame=True)

X = boston.data
y = X['CRIM']   # per capita crime rate
X = X.drop(columns=['CRIM'])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Shape:", X_train.shape, X_test.shape)
print("Columns:", X.columns.tolist())


Shape: (404, 12) (102, 12)
Columns: ['ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']


In [74]:


# Ensure numeric only
X_train_clean = X_train.apply(pd.to_numeric, errors='coerce')
y_train_clean = pd.to_numeric(y_train, errors='coerce')

# Drop rows with missing values (if any appear after conversion)
X_train_clean = X_train_clean.dropna()
y_train_clean = y_train_clean.loc[X_train_clean.index]

def best_subset_selection(X, y, max_features=13):
    results = []
    for k in range(1, max_features+1):
        for combo in combinations(X.columns, k):
            X_model = sm.add_constant(X[list(combo)].astype(float))  # force numeric
            model = sm.OLS(y.astype(float), X_model).fit()
            results.append({
                'features': combo,
                'aic': model.aic,
                'bic': model.bic,
                'adj_r2': model.rsquared_adj
            })
    results_df = pd.DataFrame(results)
    return results_df.sort_values('aic').reset_index(drop=True)

best_subsets = best_subset_selection(X_train_clean, y_train_clean, max_features=13)
print(best_subsets.head())


                              features          aic          bic    adj_r2
0        (ZN, NOX, DIS, RAD, B, LSTAT)  2708.736269  2736.746173  0.403544
1                      (RAD, B, LSTAT)  2709.046702  2725.052362  0.398698
2       (ZN, CHAS, DIS, RAD, B, LSTAT)  2709.224642  2737.234546  0.402822
3  (ZN, CHAS, NOX, DIS, RAD, B, LSTAT)  2709.224882  2741.236201  0.404270
4             (ZN, DIS, RAD, B, LSTAT)  2709.300809  2733.309299  0.401253


In [75]:
print(X_train.dtypes)
print(y_train.dtype)

ZN          float64
INDUS       float64
CHAS       category
NOX         float64
RM          float64
AGE         float64
DIS         float64
RAD        category
TAX         float64
PTRATIO     float64
B           float64
LSTAT       float64
dtype: object
float64


1. Ridge Regression

In [76]:
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

X_train = X_train.dropna()
X_test = X_test.dropna()
y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]

In [77]:
alphas = 10**np.linspace(-3, 3, 100)
ridge = RidgeCV(alphas=alphas)
ridge.fit(X_train, y_train)

ridge_pred = ridge.predict(X_test)
ridge_mse = mean_squared_error(y_test, ridge_pred)

print("Best alpha (Ridge):", ridge.alpha_)
print("Test MSE (Ridge):", ridge_mse)

Best alpha (Ridge): 0.6135907273413176
Test MSE (Ridge): 23.806517688658715


In [78]:
print(X_train.dtypes)

ZN         float64
INDUS      float64
CHAS         int64
NOX        float64
RM         float64
AGE        float64
DIS        float64
RAD          int64
TAX        float64
PTRATIO    float64
B          float64
LSTAT      float64
dtype: object


2. Lasso Regression

In [79]:
lasso = LassoCV(alphas=None, cv=10, max_iter=10000, random_state=42)
lasso.fit(X_train, y_train)

lasso_pred = lasso.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_pred)
print("Best alpha (Lasso):", lasso.alpha_)
print("Selected Features:", np.sum(lasso.coef_ != 0))
print("Test MSE (Lasso):", lasso_mse)

Best alpha (Lasso): 0.8486933195540266
Selected Features: 6
Test MSE (Lasso): 24.532353321703862


c:\Users\Kavin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1632: FutureWarning: 'alphas=None' is deprecated and will be removed in 1.9, at which point the default value will be set to 100. Set 'alphas=100' to silence this warning.
  warnings.warn(


3. Principal Component Regression (PCR)

In [80]:
mse_pcr = []
for n in range(1, X_train.shape[1]+1):
    pcr = make_pipeline(StandardScaler(), PCA(n_components=n), LinearRegression())
    scores = cross_val_score(pcr, X_train, y_train, scoring='neg_mean_squared_error', cv=10)
    mse_pcr.append(-scores.mean())

best_ncomp = np.argmin(mse_pcr) + 1
print("Best #components:", best_ncomp)

pcr_final = make_pipeline(StandardScaler(), PCA(n_components=best_ncomp), LinearRegression())
pcr_final.fit(X_train, y_train)
pcr_pred = pcr_final.predict(X_test)
pcr_mse = mean_squared_error(y_test, pcr_pred)
print("Test MSE (PCR):", pcr_mse)

Best #components: 12
Test MSE (PCR): 23.864882518234275


 (b) Propose a model (or set of models) that seem to perform well on this data set, and justify your answer. Make sure that you are evaluating model performance using validation set error, crossvalidation, or some other reasonable alternative, as opposed to using training error.

In [81]:
X = boston.data
y = X['CRIM']   
X = X.drop(columns=['CRIM'])

X = X.apply(pd.to_numeric, errors="coerce")

X_train, X_test, y_train, y_test = train_test_split(
    X.astype(float), y.astype(float), test_size=0.2, random_state=42
)

cv = KFold(n_splits=10, shuffle=True, random_state=42)

1. Linear Regression (baseline)

In [82]:
linreg = LinearRegression()
linreg_scores = cross_val_score(linreg, X_train, y_train,
                                scoring='neg_mean_squared_error', cv=cv)
linreg_mse_cv = -linreg_scores.mean()

linreg.fit(X_train, y_train)
linreg_mse_test = mean_squared_error(y_test, linreg.predict(X_test))


2. Ridge Regression

In [83]:
alphas = 10**np.linspace(-3, 3, 100)
ridge = RidgeCV(alphas=alphas, cv=cv, scoring='neg_mean_squared_error')
ridge.fit(X_train, y_train)

ridge_scores = cross_val_score(ridge, X_train, y_train,
                               scoring='neg_mean_squared_error', cv=cv)
ridge_mse_cv = -ridge_scores.mean()

ridge_mse_test = mean_squared_error(y_test, ridge.predict(X_test))

3. Lasso Regression

In [84]:
lasso = LassoCV(alphas=None, cv=cv, max_iter=10000, random_state=42)
lasso.fit(X_train, y_train)

lasso_mse_cv = np.min(lasso.mse_path_.mean(axis=1))
lasso_mse_test = mean_squared_error(y_test, lasso.predict(X_test))
selected_features = np.sum(lasso.coef_ != 0)

c:\Users\Kavin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:1632: FutureWarning: 'alphas=None' is deprecated and will be removed in 1.9, at which point the default value will be set to 100. Set 'alphas=100' to silence this warning.
  warnings.warn(


4. Principal Component Regression (PCR)

In [ ]:
mse_pcr = []
for n in range(1, X_train.shape[1] + 1):
    pcr = make_pipeline(StandardScaler(), PCA(n_components=n), LinearRegression())
    scores = cross_val_score(pcr, X_train, y_train,
                             scoring='neg_mean_squared_error', cv=cv)
    mse_pcr.append(-scores.mean())

best_ncomp = np.argmin(mse_pcr) + 1
pcr_final = make_pipeline(StandardScaler(), PCA(n_components=best_ncomp), LinearRegression())
pcr_final.fit(X_train, y_train)
pcr_mse_test = mean_squared_error(y_test, pcr_final.predict(X_test))

print("====== Cross-Validation MSE (Train) ======")
print(f"Linear Regression: {linreg_mse_cv:.4f}")
print(f"Ridge Regression: {ridge_mse_cv:.4f} (alpha={ridge.alpha_:.4f})")
print(f"Lasso Regression: {lasso_mse_cv:.4f} (alpha={lasso.alpha_:.4f})")
print(f"PCR (best {best_ncomp} comps): {min(mse_pcr):.4f}")

print("\n====== Test Set MSE ======")
print(f"Linear Regression: {linreg_mse_test:.4f}")
print(f"Ridge Regression: {ridge_mse_test:.4f}")
print(f"Lasso Regression: {lasso_mse_test:.4f} (selected {selected_features} features)")
print(f"PCR (best {best_ncomp} comps): {pcr_mse_test:.4f}")

====== Cross-Validation MSE (Train) ======
Linear Regression: 47.7620
Ridge Regression: 47.9502 (alpha=0.1748)
Lasso Regression: 48.2221 (alpha=0.8487)
PCR (best 12 comps): 47.7620

====== Test Set MSE ======
Linear Regression: 23.8649
Ridge Regression: 23.8396
Lasso Regression: 24.5324 (selected 6 features)
PCR (best 12 comps): 23.8649


 (c) Does your chosen model involve all of the features in the data set? Why or why not?

Interpretation:

Linear Regression: Yes, it uses all features in X_train. Standard linear regression does not perform feature selection by itself, so every column contributes to the prediction.

Ridge Regression: Yes, it also uses all features, but the coefficients are shrunk toward zero to reduce overfitting. Ridge does not set any coefficient exactly to zero—it only regularizes them.

Lasso Regression: No, Lasso performs feature selection. It shrinks some coefficients exactly to zero depending on the value of alpha. In your case, selected_features = 6, meaning only 6 features out of the original set contribute to the final model. The rest are ignored.

PCR (Principal Component Regression): Indirectly, PCR transforms all features into principal components first. The model then uses the best n components (here, 12 components) instead of the raw features. While all original features contribute to the components, the regression effectively reduces the dimensionality, so it does not use all features directly.


Why or why not?

Ridge: Uses all features because it only shrinks coefficients to reduce variance, not to eliminate predictors.

Lasso: May ignore some features because its L1 penalty encourages sparsity, which can improve interpretability and reduce overfitting.

PCR: Reduces dimensionality by combining correlated features into components, which captures most of the variance while ignoring less important directions.